In [1]:

import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import PIL
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import LeakyReLU, BatchNormalization, ReLU, Activation
from tensorflow.keras.layers import UpSampling2D, Conv2D, Concatenate, Dense, concatenate
from tensorflow.keras.layers import Flatten, Lambda, Reshape, ZeroPadding2D, add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation
from keras.models import Model
from tensorflow.keras.utils import to_categorical
import keras.backend as K

import warnings
warnings.filterwarnings("ignore")
import pandas as pd2
import numpy as np
from sklearn.datasets import load_linnerud
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion
from physlearn import Regressor
import pandas as pd


physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [26]:
# Загрузим данные. Я их немножно подправил ещё до загрузки.
df = pd.read_excel("data_00.xlsx")

# Уберём ненужные столбцы и пустые значения
df = df.drop(columns=["Номер образца", "Адсорбент"]).dropna()

# Ещё один нюанс
df = df[df["Sme, м2/г"] != "-"]

# Сменим тип данных у таблицы
df["Sme, м2/г"] = df["Sme, м2/г"].astype(np.float64, copy=False)
df["m (соли), г"] = df["m (соли), г"].astype(np.float64, copy=False)
df["Vпр. (р-ля), мл"] = df["Vпр. (р-ля), мл"].astype(np.float64, copy=False)

# У нас есть 3 категориальных признака, которых мы закодируем числами
list_of_cats = ["Металл", "Лиганд", "Растворитель"]
cat2id = {cat:{v:i for i, v in enumerate(df[cat].drop_duplicates().values)} for cat in list_of_cats}
id2cat = {cat:{i:v for i, v in enumerate(df[cat].drop_duplicates().values)} for cat in list_of_cats}
for cat in list_of_cats:
    df[cat] = df[cat].apply(lambda x: cat2id[cat][x])
    


## GANs implementation

In [3]:
# HIDE OUTPUT
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

In [4]:
# Будем предсказывать 2 набора данных:
# 1) Параметры МОК
# 2) Параметры синтеза МОК

In [22]:
# Кусок кода взят отсюда: https://scikit-physlearn.readthedocs.io/en/latest/quick_start.html

bunch = load_linnerud(as_frame=True)  # returns a Bunch instance
X, y = df.iloc[:,:10], df.iloc[:,10:]
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=42)

transformer_list = [('pca', PCA(n_components=1)),
                    ('svd', TruncatedSVD(n_components=2))]
union = FeatureUnion(transformer_list=transformer_list, n_jobs=-1)

# Select a regressor, e.g., LGBMRegressor from LightGBM,
# with a case-insensitive string.
reg = Regressor(regressor_choice='lgbmregressor',
                pipeline_transform=('tr', union),
                scoring='neg_mean_absolute_error')

# Automatically build the pipeline with final estimator MultiOutputRegressor
# from Sklearn, then exhaustively search over the (hyper)parameters.
search_params = dict(reg__boosting_type=['gbdt', 'goss'],
                     reg__n_estimators=[6, 8, 10, 20])
reg.search(X_train, y_train, search_params=search_params,
           search_method='gridsearchcv')

# Generate predictions with the refit regressor, then
# compute the average mean absolute error.
y_pred = reg.fit(X_train, y_train).predict(X_test)
score = reg.score(y_test, y_pred)
print(round(score['mae'].mean(),2))
# print(score['mae'].mean().round(decimals=2))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[Pipeline] ................ (step 1 of 2) Processing tr, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing reg, total=   0.0s
[Pipeline] ................ (step 1 of 2) Processing tr, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing reg, total=   0.1s
13.87


In [11]:
y_pred.head()

,0,1,2,3,4,5,6,7,8,9
236,2.965634,0.740797,0.066413,6.981222,3.523975,75.053621,115.490869,364.656989,114.577164,6.893374
9,0.756878,1.127329,0.010475,1.435568,0.805628,31.810824,130.511127,163.634979,128.440354,1.214902
57,1.885257,1.532047,0.009619,2.232438,1.193876,33.055860,124.615397,149.844659,131.401259,2.522574
60,2.331198,0.913095,0.109679,2.481090,1.684954,56.359493,127.587932,211.413984,126.211331,2.912554
25,1.731507,1.152496,0.014310,1.959854,1.368838,54.442805,133.507185,193.249978,135.264355,2.246213


In [7]:
# Раскодируем наши категориальные признаки
y_pred.columns = y.columns

for cat in list_of_cats:
    y_pred.loc[:,cat] = y_pred[cat].apply(lambda x: id2cat[cat][round(x)])

# Сохраним результат
y_pred.to_excel("baseline_results.xlsx", index=False)

In [24]:
from tabgan.sampler import GANGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# for step in range(10):

gen_x, gen_y = GANGenerator(gen_x_times=1.1, cat_cols=None,
           bot_filter_quantile=0.001, top_filter_quantile=0.999, \
              is_post_process=True,
           adversarial_model_params={
               "metrics": "rmse", "max_depth": 2, "max_bin": 100, 
               "learning_rate": 0.02, "random_state": \
                42, "n_estimators": 500,
           }, pregeneration_frac=2, only_generated_data=False,\
           gan_params = {"batch_size": 500, "patience": 25, \
          "epochs" : 500,}).generate_data_pipe(X_train,pd.DataFrame(data=y_train.iloc[:,0]) ,\
          X_test, deep_copy=True, only_adversarial=False, \
          use_adversarial=True)

Training CTGAN, epochs::  13%|█▎        | 64/500 [00:05<00:39, 11.05it/s]


LightGBMError: Do not support special JSON characters in feature name.

In [27]:
pd.DataFrame(data=y_train.iloc[:,0])

,
119,1
266,4
252,5
212,0
137,1
...,...
195,4
71,1
106,4
278,4


In [ ]:
# Predict
score = np.sqrt(metrics.mean_squared_error(y_pred.values,gen_y.values))
print("Final score (RMSE): {}".format(score))